# Documentation: Retrieving Historical Stock Data for S&P 500 Tickers Using EODHD API

---

## Overview

This Python script retrieves historical end-of-day stock data for all S&P 500 tickers from January 1, 2019, to the present using the EODHD API. The data includes the following fields for each trading day:

- `date`
- `open`
- `high`
- `low`
- `close`
- `adjusted_close`
- `volume`

The script processes each ticker individually, making one API call per ticker, and saves the data into separate CSV files (e.g., `AAPL_full_data.csv`) within a folder named `sp500_full_data`. This approach ensures that the data is organized and easily accessible for individual stocks.

---

## Prerequisites

Before running the script, ensure you have the following:

- **EODHD API Key**: Obtain a valid API key from [EODHD](https://eodhd.com/). You will need to replace `"YOUR_EODHD_API_KEY"` in the script with your actual key.
- **Python Environment**: The script requires Python 3.x and the following libraries:
  - `pandas`
  - `requests`
  - `os` (standard library)
  - `csv` (standard library)
  - `datetime` (standard library)
- **Ticker List**: A CSV file named `sp500_constituents.csv` containing the S&P 500 ticker symbols in a column named `Ticker`. This file must be placed in the same directory as the script.

---

## Setup

1. **Install Required Libraries**  
   If not already installed, use pip to install the necessary Python libraries:
   ```bash
   pip install pandas requests
   ```
   Note: The `os`, `csv`, and `datetime` modules are part of Python's standard library and do not require installation.

2. **Prepare the Ticker List**  
   Ensure that `sp500_constituents.csv` is correctly formatted with a `Ticker` column. Example format:
   ```
   Ticker,Company,Sector,CIK
   MMM,3M Company,Industrials,66740
   AOS,A. O. Smith Corporation,Industrials,91142
   ABT,Abbott Laboratories,Health Care,1800
   ...
   ```

3. **Insert API Key**  
   Open the script and replace `"YOUR_EODHD_API_KEY"` with your actual EODHD API key.

---

## How the Script Works

The script performs the following steps:

1. **Load Ticker List**  
   Reads the `Ticker` column from `sp500_constituents.csv` to obtain the list of approximately 505 S&P 500 tickers.

2. **Define Date Range**  
   Sets the start date to January 1, 2019, and the end date to the current date using `datetime.today()`.

3. **Create Output Folder**  
   Creates a folder named `sp500_full_data` to store the CSV files for each ticker. If the folder already exists, it is reused without modification.

4. **Process Each Ticker**  
   - Constructs the API URL for the EODHD `eod` endpoint, specifying the ticker, date range, API key, and CSV format.
   - Sends an API request to fetch the historical data for the ticker.
   - If the request succeeds (HTTP status code 200), parses the CSV response.
   - Saves the data to a file named `<ticker>_full_data.csv` (e.g., `AAPL_full_data.csv`) in the `sp500_full_data` folder, including all fields: `date`, `open`, `high`, `low`, `close`, `adjusted_close`, and `volume`.

5. **Error Handling**  
   If an API request fails (e.g., due to an invalid ticker or network issues), the script prints an error message with the ticker and status code, then continues processing the remaining tickers.

6. **API Rate Limits**  
   The script makes one API call per ticker (approximately 505 calls). With EODHD's limits of 1000 calls per minute and 100,000 calls per day, this usage is well within bounds, requiring no delays.

---

## Output

- **Folder**: All files are saved in the `sp500_full_data` folder.
- **Files**: Each ticker has its own CSV file, named `<ticker>_full_data.csv` (e.g., `MMM_full_data.csv`).
- **File Structure**: Each CSV contains:
  - `date`: Trading day (e.g., `2019-01-02`)
  - `open`: Opening price
  - `high`: Highest price
  - `low`: Lowest price
  - `close`: Closing price
  - `adjusted_close`: Adjusted closing price (accounts for splits, dividends, etc.)
  - `volume`: Shares traded

- **Sample Data** (from `AAPL_full_data.csv`):
  ```
  date,open,high,low,close,adjusted_close,volume
  2019-01-02,38.72,39.71,38.56,39.48,38.27,37039700
  2019-01-03,35.55,36.43,35.50,35.55,34.46,91312200
  ...
  2023-01-03,118.47,118.80,104.64,108.10,108.10,231402797
  ...
  ```

- **Row Count**: Approximately 1,200 rows per file, reflecting trading days from 2019 to the present.

---

## Usage Instructions

1. **Prepare the Environment**  
   - Install Python and required libraries.
   - Place `sp500_constituents.csv` in the script’s directory.
   - Insert your EODHD API key into the script.

2. **Run the Script**  
   Execute the script from the command line:
   ```bash
   python script_name.py
   ```
   - The script will process each ticker, display progress, and save data to `sp500_full_data`.
   - Completion takes approximately 4-5 minutes, depending on network speed.

3. **Verify the Output**  
   - Check the `sp500_full_data` folder for CSV files.
   - Open a file (e.g., `MMM_full_data.csv`) to confirm it contains the expected data.

---

## API Usage and Rate Limits

- **Total API Calls**: Approximately 505 calls (one per ticker).
- **Rate Limits**:
  - **Per Minute**: 1000 calls allowed; 505 calls are within this limit.
  - **Per Day**: 100,000 calls allowed; 505 calls use only 0.505% of the daily limit.
- **Shared Accounts**: If the API key is shared, coordinate with others to stay under 100,000 daily calls. This script’s usage is minimal and unlikely to cause conflicts.

---

## Additional Notes

- **Testing**: For a smaller test run, modify the ticker list (e.g., `tickers = tickers[:5]`).
- **Error Handling**: Failed requests are logged to the console; the script continues running.
- **Data Updates**: To update existing files, adjust `start_date` to the last recorded date and append new data.

This documentation provides a complete guide to setting up, running, and understanding the script’s output for retrieving S&P 500 historical stock data.

In [1]:
import pandas as pd
import requests
import csv
import os
from datetime import datetime

# Replace with your EODHD API key
API_KEY = "603eafd117ce29.12741264"

# Load S&P 500 tickers from a CSV file (assumed to have a 'Ticker' column)
df = pd.read_csv("sp500_constituents.csv")
tickers = df["Ticker"].tolist()  # Approximately 505 tickers

# Define the date range
start_date = "2019-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")  # Today's date

# Create a folder to store the CSV files
folder_name = "sp500_prices"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Process each ticker
for i, ticker in enumerate(tickers):
    # Construct the API URL
    url = (
        f"https://eodhd.com/api/eod/{ticker}.US?"
        f"from={start_date}&"
        f"to={end_date}&"
        f"api_token={API_KEY}&"
        f"fmt=csv"
    )
    
    # Fetch the data
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the CSV response
        lines = response.text.splitlines()
        # Save to a ticker-specific file in the folder
        file_path = os.path.join(folder_name, f"{ticker}_prices.csv")
        with open(file_path, mode="w", newline="") as f:
            writer = csv.writer(f)
            # Write the header (first line of the response)
            writer.writerow(lines[0].split(","))
            # Write the data rows
            for line in lines[1:]:
                writer.writerow(line.split(","))
        print(f"Processed {ticker} ({i+1}/{len(tickers)})")
    else:
        print(f"Failed to fetch {ticker}: {response.status_code}")

print(f"All tickers processed. Data saved in individual CSV files inside '{folder_name}' folder.")

Processed MMM (1/503)
Processed AOS (2/503)
Processed ABT (3/503)
Processed ABBV (4/503)
Processed ACN (5/503)
Processed ADBE (6/503)
Processed AMD (7/503)
Processed AES (8/503)
Processed AFL (9/503)
Processed A (10/503)
Processed APD (11/503)
Processed ABNB (12/503)
Processed AKAM (13/503)
Processed ALB (14/503)
Processed ARE (15/503)
Processed ALGN (16/503)
Processed ALLE (17/503)
Processed LNT (18/503)
Processed ALL (19/503)
Processed GOOGL (20/503)
Processed GOOG (21/503)
Processed MO (22/503)
Processed AMZN (23/503)
Processed AMCR (24/503)
Processed AEE (25/503)
Processed AEP (26/503)
Processed AXP (27/503)
Processed AIG (28/503)
Processed AMT (29/503)
Processed AWK (30/503)
Processed AMP (31/503)
Processed AME (32/503)
Processed AMGN (33/503)
Processed APH (34/503)
Processed ADI (35/503)
Processed ANSS (36/503)
Processed AON (37/503)
Processed APA (38/503)
Processed APO (39/503)
Processed AAPL (40/503)
Processed AMAT (41/503)
Processed APTV (42/503)
Processed ACGL (43/503)
Proces

## Running the code for BRK-B and BF-B

In [2]:
import requests
import csv
import os
from datetime import datetime

# Replace with your EODHD API key
API_KEY = "603eafd117ce29.12741264"

# Hardcoded list of tickers
tickers = ["BRK-B", "BF-B"]

# Define the date range
start_date = "2019-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")  # Today's date

# Create a folder to store the CSV files
folder_name = "sp500_prices"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Process each ticker
for i, ticker in enumerate(tickers):
    # Construct the API URL
    url = (
        f"https://eodhd.com/api/eod/{ticker}.US?"
        f"from={start_date}&"
        f"to={end_date}&"
        f"api_token={API_KEY}&"
        f"fmt=csv"
    )

    # Fetch the data
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the CSV response
        lines = response.text.splitlines()

        # Save to a ticker-specific file in the folder
        file_path = os.path.join(folder_name, f"{ticker}_prices.csv")
        with open(file_path, mode="w", newline="") as f:
            writer = csv.writer(f)
            # Write the header (first line of the response)
            writer.writerow(lines[0].split(","))
            # Write the data rows
            for line in lines[1:]:
                writer.writerow(line.split(","))
        print(f"Processed {ticker} ({i+1}/{len(tickers)})")
    else:
        print(f"Failed to fetch {ticker}: {response.status_code}")

print(f"All tickers processed. Data saved in individual CSV files inside '{folder_name}' folder.")

Processed BRK-B (1/2)
Processed BF-B (2/2)
All tickers processed. Data saved in individual CSV files inside 'sp500_prices' folder.
